In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import pandas as pd
from keras_preprocessing.image import ImageDataGenerator
import tensorflow_addons as tfa

In [2]:
directory = 'C:\datasets\img_align_celeba\img_align_celeba'

In [3]:
tag = pd.read_csv('C:/datasets/list_attr_celeba.csv')

In [4]:
col = ['image_id', 'Male', 'Mustache', 'Bald', 'Smiling', 'Young']
tag = tag[col]
tag = tag.replace(-1, 0)
col = col[1:]
tag

,image_id,Male,Mustache,Bald,Smiling,Young
0,000001.jpg,0,0,0,1,1
1,000002.jpg,0,0,0,1,1
2,000003.jpg,1,0,0,0,1
3,000004.jpg,0,0,0,0,1
4,000005.jpg,0,0,0,0,1
...,...,...,...,...,...,...
202594,202595.jpg,0,0,0,0,1
202595,202596.jpg,1,0,0,1,1
202596,202597.jpg,1,0,0,1,1
202597,202598.jpg,0,0,0,1,1


In [5]:
train = tag.sample(frac = .8, random_state=42)
test = tag.drop(train.index)
train

,image_id,Male,Mustache,Bald,Smiling,Young
93241,093242.jpg,1,0,0,0,1
121413,121414.jpg,0,0,0,0,1
10624,010625.jpg,1,0,0,0,1
82523,082524.jpg,0,0,0,1,1
73192,073193.jpg,0,0,0,1,0
...,...,...,...,...,...,...
36890,036891.jpg,1,0,0,0,1
1992,001993.jpg,0,0,0,0,1
47852,047853.jpg,0,0,0,1,1
79821,079822.jpg,0,0,0,1,1


In [6]:
datagen=ImageDataGenerator(rescale=1./255.,validation_split=0.25)

In [7]:
train_generator=datagen.flow_from_dataframe(
dataframe=train,
directory = directory,
x_col="image_id",
y_col=col,
subset="training",
batch_size=16,
seed=42,
shuffle=True,
    class_mode='raw',
target_size=(256,256))

valid_generator=datagen.flow_from_dataframe(
dataframe=train,
directory=directory,
x_col="image_id",
y_col=col,
subset="validation",
batch_size=16,
seed=42,
shuffle=True,
class_mode="raw",
target_size=(256,256))

test_datagen=ImageDataGenerator(rescale=1./255.)
test_generator=test_datagen.flow_from_dataframe(
dataframe=test,
directory=directory,
x_col="image_id",
y_col=col,
batch_size=16,
seed=42,
shuffle=False,
class_mode='raw',
target_size=(256,256))

Found 121560 validated image filenames.
Found 40519 validated image filenames.
Found 40520 validated image filenames.


In [8]:
class Spatial_Gating_Unit(tf.keras.layers.Layer):
    def __init__(self, patch_size:int, initial_stddev:float):
        super(Spatial_Gating_Unit, self).__init__()
        self.patch_size = patch_size
        self.initial_stddev = initial_stddev
        
        self.ln = tf.keras.layers.LayerNormalization()
        self.t = tf.keras.layers.Permute((2,1))
        self.Wb = tf.keras.layers.Dense(int(self.patch_size*self.patch_size), kernel_initializer = tf.keras.initializers.RandomNormal(stddev=self.initial_stddev), bias_initializer = 'ones')
        self.M = tf.keras.layers.Multiply()
        
    def call(self, X):
        z1, z2 = tf.split(X, 2, axis=-1)
        z2 = self.ln(z2)
        z2 = self.t(z2)
        z2 = self.Wb(z2)
        z2 = self.t(z2)
        X = self.M([z1,z2])
        return X
        
        
class gMLPs_Block(tf.keras.layers.Layer):
    def __init__(self, d_model:int, patch_size:int, initial_stddev:float, d_ffn:int, survival_prob:float):
        super(gMLPs_Block, self).__init__()
        self.d_model = d_model
        self.patch_size = patch_size
        self.initial_stddev = initial_stddev
        self.d_ffn = d_ffn
        self.survival_prob = survival_prob
        
        self.ln = tf.keras.layers.LayerNormalization()
        self.U = tf.keras.layers.Dense(self.d_ffn, activation = 'gelu', kernel_initializer = tf.keras.initializers.lecun_normal())
        self.SGU = Spatial_Gating_Unit(self.patch_size, self.initial_stddev)
        self.V = tf.keras.layers.Dense(self.d_model, kernel_initializer = tf.keras.initializers.glorot_normal())
        
    def call(self, X):
        y = self.ln(X)
        y = self.U(y)
        y = self.SGU(y)
        y = self.V(y)
        y = y * tf.keras.backend.random_bernoulli(shape = (1,), p = self.survival_prob)
        y = X + y
        # U : [batchs, patch_size^2, d_ffn] -> SGU : [batch, patch_size^2, d_ffn/2] -> V : [batch, patch_size^2, d_models] 
        return y
    
class gMLPs(tf.keras.models.Model):
    def __init__(self, d_model:int, d_ffn:int, image_size:int, patch_size:int, n_res_layers:int, n_labels:int,
                 survival_prob = 1., initial_stddev:float = 0.00001, mode:str = 'softmax'):
        super(gMLPs, self).__init__()
        self.d_model = d_model
        self.d_ffn = d_ffn
        self.image_size = image_size
        self.patch_size = patch_size
        if (self.image_size % self.patch_size) != 0:
            raise ValueError('size error')
        self.n_patches = int((tf.square(self.image_size) / tf.square(self.patch_size)).numpy())
        self.n_res_layers = n_res_layers
        self.n_labels = n_labels
        self.initial_stddev = initial_stddev
        if mode not in ['sigmoid','softmax']:
            raise ValueError('mode must be sigmoid or softmax')
        else:
            self.mode = mode
        self.survival_prob = survival_prob
        
        self.patchConv = tf.keras.layers.Conv2D(self.d_model, (self.patch_size, self.patch_size), strides = (self.patch_size, self.patch_size))
        self.reshapeL = tf.keras.layers.Reshape((self.n_patches, self.d_model,))
        self.gMLPBlocks = [gMLPs_Block(self.d_model, self.patch_size, self.initial_stddev, self.d_ffn) for x in range(self.n_res_layers)]
        self.gap = tf.keras.layers.GlobalAveragePooling2D()
        self.classifier = tf.keras.layers.Dense(self.n_labels if self.n_labels > 2 else 1, activation = mode, kernel_initializer = tf.keras.initializers.glorot_normal(seed = 42))
        
    def call(self, X):
        X = self.patchConv(X)
        X = self.reshapeL(X)
        for gMLPB in self.gMLPBlocks:
            X = gMLPB(X)
        X = self.gap(X)
        X = self.classifier(X)
        return X

In [9]:
gmlp = gMLPs(128, 768, 256, 16, 30, len(col), mode = 'sigmoid')
opt = tf.keras.optimizers.Adam(.00005)
gmlp.compile(optimizer=opt, loss=tfa.losses.focal_loss.sigmoid_focal_crossentropy, metrics='acc')

In [10]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
gmlp.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=10)

C:\Users\soymi\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
7597/7597 [==============================] - 1161s 151ms/step - loss: 0.1426 - acc: 0.6961 - val_loss: 0.0998 - val_acc: 0.8018
Epoch 2/10
7597/7597 [==============================] - 1151s 152ms/step - loss: 0.1019 - acc: 0.7830 - val_loss: 0.1050 - val_acc: 0.8216
Epoch 3/10
7597/7597 [==============================] - 1153s 152ms/step - loss: 0.0891 - acc: 0.8079 - val_loss: 0.0937 - val_acc: 0.7378
Epoch 4/10
7597/7597 [==============================] - 1151s 151ms/step - loss: 0.0793 - acc: 0.8255 - val_loss: 0.0857 - val_acc: 0.8194
Epoch 5/10
7597/7597 [==============================] - 1154s 152ms/step - loss: 0.0697 - acc: 0.8391 - val_loss: 0.0915 - val_acc: 0.8535
Epoch 6/10
7597/7597 [==============================] - 1158s 152ms/step - loss: 0.0585 - acc: 0.8486 - val_loss: 0.0881 - val_acc: 0.8194
Epoch 7/10
7597/7597 [==============================] - 1151s 151ms/step - loss: 0.0452 - acc: 0.8502 - val_loss: 0.1037 - val_acc: 0.8182
Epoch 8/10
7597/7597 [=====